第十二章继承的优缺点

12.1  子类化内置类型很麻烦

In [1]:
#基本上，内置类型不会调用子类覆盖的方法
#内置类型的dict的__init__和__update__方法会忽略我们覆盖的__setitem__方法
class DoppelDict(dict):
    def __setitem__(self,key,value):
        super().__setitem__(key,[value]*2)
dd=DoppelDict(one=1)

In [2]:
dd

{'one': 1}

In [3]:
dd['two']=2

In [4]:
dd

{'one': 1, 'two': [2, 2]}

In [5]:
dd.update(three=3)
dd

{'one': 1, 'three': 3, 'two': [2, 2]}

不只是实例内部有这个问题(self.get()不调用self.__getitem__()）,内置类型的方法调用的其他类的方法，如果被覆盖了，也不会被调用

In [6]:
#dict.update方法会忽略AnswerDict.__getitem__方法
class AnswerDict(dict):
    def __getitem__(self, item):
        return  42
ad=AnswerDict(a='foo')
ad['a']#符合预期

42

In [9]:
dd=AnswerDict(one=1)
dd #会调用__getitem__方法

{'one': 42}

In [10]:
dd.update(three=3)
dd#同上

{'one': 42, 'three': 42}

In [11]:
d={}
d.update(ad)#d是dict的实例，使用ad中的值更新d
d['a']#dict.update方法忽略了AnswerDict.__getitem__方法


'foo'

不要子类化内置类型！用户自己定义的类应该继承collection模块UserDict,UserList,UserString

In [12]:
import collections
class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key,[value]*2)
dd=DoppelDict2(one=1)
dd

{'one': [1, 1]}

In [13]:
dd['two']=2
dd

{'one': [1, 1], 'two': [2, 2]}

In [15]:
dd.update(three=3)
dd

{'one': [1, 1], 'three': [3, 3], 'two': [2, 2]}

In [17]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, item):
        return  42
ad=AnswerDict2(a='foo')
ad['a']

42

In [18]:
ad

{'a': 'foo'}

In [19]:
d={}
d.update(ad)
d['a']

42

In [20]:
d

{'a': 42}

12.2 多重继承和方法解析顺序

In [21]:
#菱形问题：多重继承，由不相关的祖先类实现同名方法引起命名冲突
class A:
    def ping(self):
        print('ping:', self)


class B(A):
    def pong(self):
        print('pong:', self)


class C(A):
    def pong(self):
        print('PONG:', self)


class D(B, C):

    def ping(self):
        super().ping()
        print('post-ping:', self)

    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)

In [22]:
d=D()
d.pong() #直接调用d.pong()运行的是B类中的版本

pong: <__main__.D object at 0x0000017F45ABF048>


In [23]:
C.pong(d) #超类中的方法可以直接调用，此时要把实例作为显式参数传入

PONG: <__main__.D object at 0x0000017F45ABF048>


In [25]:
D.__mro__#元组，按照解析顺序列出各个超类

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [28]:
#若想把方法委托给超类，推荐使用内置的super()函数，如D类的pingpong方法，但有时需要绕过方法解析顺序,D.ping可以这样写
def ping(self):
    A.ping(self) #而不是super().ping()
    print('post-ping',self)


In [29]:
d.ping()
#D类的ping方法做了两次调用
#第一个调用时super().ping();super函数把ping调用委托给A类，这一行有A.ping输出
#第二个调用时print('post-ping',self)

ping: <__main__.D object at 0x0000017F45ABF048>
post-ping: <__main__.D object at 0x0000017F45ABF048>


In [30]:
d.pingpong()

ping: <__main__.D object at 0x0000017F45ABF048>
post-ping: <__main__.D object at 0x0000017F45ABF048>
ping: <__main__.D object at 0x0000017F45ABF048>
pong: <__main__.D object at 0x0000017F45ABF048>
pong: <__main__.D object at 0x0000017F45ABF048>
PONG: <__main__.D object at 0x0000017F45ABF048>


In [31]:
#共5个调用
#第一个是self.ping()输出第一行和第二行
#第二个是super().ping()，跳过D类的ping方法，找到A类的ping方法
#第三个是self.pong()，根据__mro__，找到的是B类实现的pong方法
#第四个是super().pong(),也根据__mro__找到B的pong方法
#第五个是C.pong(self),忽略__mro__，找到C的pong方法

In [32]:
#如果D类声明为class D(C,B):，那么D类的__mro__属性会不一样：先搜索C，再搜素B.分析类时，可先查看__mro__属性
bool.__mro__

(bool, int, object)

In [33]:
list.__mro__

(list, object)

In [34]:
import numbers
numbers.Integral.__mro__

(numbers.Integral,
 numbers.Rational,
 numbers.Real,
 numbers.Complex,
 numbers.Number,
 object)

In [35]:
import io
io.BytesIO.__mro__#后缀为Base的是抽象基类

(_io.BytesIO, _io._BufferedIOBase, _io._IOBase, object)

In [36]:
import tkinter
tkinter.Text.__mro__

(tkinter.Text,
 tkinter.Widget,
 tkinter.BaseWidget,
 tkinter.Misc,
 tkinter.Pack,
 tkinter.Place,
 tkinter.Grid,
 tkinter.XView,
 tkinter.YView,
 object)